In [1]:
import os
from datetime import datetime
import json
import lyricsgenius as lg
import pandas as pd

In [2]:
artist_name = "Zamdane"

In [11]:
genius = lg.Genius(access_token=os.environ['GENIUS_CLIENT_ACCESS_TOKEN'],
                   excluded_terms=["(Remix)", "(Live)", "(Freestyle)", "(Instrumental)"],
                   retries=10,
                   timeout=60)

genius_search_artist_result_dict = genius.search_artist(artist_name,
                                                        max_songs=999,
                                                        get_full_info=True,
                                                        allow_name_change=False).to_dict()

Searching for songs by Zamdane...

Song 1: "Favaro"
Song 2: "Affamé #8 - Hayati"
Song 3: "Boyka"
Song 4: "Disaster"
"Noir (Remix)" is not valid. Skipping.
Song 5: "Le monde par ma fenêtre"
"Règlement Freestyle #6" is not valid. Skipping.
Song 6: "Vide quand t’es pas là"
Song 7: "Affamé #16 - Triste mais elle aime ça"
Song 8: "Mouchkila"
Song 9: "Quand j’fume"
Song 10: "1,2,3,4"
Song 11: "Flouka"
"Freestyle #11RappeursASuivre2022" is not valid. Skipping.
Song 12: "Affamé #1 - Shook"
Song 13: "Tout ce qu’il voulait"
Song 14: "Zhar"
Song 15: "Stradivarius"
Song 16: "Minato"
Song 17: "Affamé #10 - Athéna"
Song 18: "Naïf"
Song 19: "TJPC"
Song 20: "Deadstar"
Song 21: "Affamé #2 - 21"
Song 22: "Fauves"
Song 23: "Fallait"
Song 24: "Million"
Song 25: "Alouette"
Song 26: "Youm wara youm"
Song 27: "Noum"
Song 28: "Foust L’Bando"
Song 29: "Monstres"
Song 30: "Doko"
Song 31: "Nil"
Song 32: "Fleurs"
Song 33: "Affamé #12 - Sentimental"
Song 34: "Affamé #13 - Marseille"
Song 35: "Padmé"
Song 36: "Gree

In [5]:
from typing import Dict, Any

# Define a constant for unknown values
UNKNOWN_VALUE = 'UNKNOWN'

def extract_value(data: Dict[str, Any], key_path: list, UNKNOWN_VALUE=UNKNOWN_VALUE) -> Any:
    """
    Extracts a value from a nested dictionary based on the provided list of keys (key_path).
    
    This function traverses the dictionary `data` using the keys in `key_path`. If any key along the path is not 
    present or the value is not found, it returns the `UNKNOWN_VALUE` constant. It also handles cases where a 
    non-dictionary value is encountered in the middle of the path.
    
    Args:
        data (Dict[str, Any]): The dictionary from which the value will be extracted.
        key_path (list): A list of keys representing the path to the target value.
        UNKNOWN_VALUE (str): The value to return if any key in the path is missing. Defaults to 'UNKNOWN'.
    
    Returns:
        Any: The value found at the specified key path, or 'UNKNOWN' if not found.
    """
    
    value = data

    try:
        for key in key_path:
            value = value.get(key, UNKNOWN_VALUE)
            if value == UNKNOWN_VALUE:
                break
    except AttributeError:
        return UNKNOWN_VALUE
    
    return value if value else UNKNOWN_VALUE

In [6]:
from typing import Dict, Any

def build_artist_song_genius(raw_data_artist_song_dict: Dict[str, Any], UNKNOWN_VALUE: Any = 'UNKNOWN') -> Dict[str, Any]:
    """
    Extracts and structures song information from the raw data retrieved from Genius.

    This function processes a nested dictionary (`raw_data_artist_song_dict`) representing song information 
    from Genius and returns a dictionary containing relevant details like song ID, title, release date, 
    album information, pageviews, and artist information. If any key is missing, it defaults to `UNKNOWN_VALUE`.

    Args:
        raw_data_artist_song_dict (Dict[str, Any]): Raw data dictionary containing song information.
        UNKNOWN_VALUE (Any): The value to return when required data is missing. Defaults to 'UNKNOWN'.

    Returns:
        Dict[str, Any]: A dictionary containing extracted song details such as title, album info, artist info, and more.
    """

    def safe_get(data: Dict[str, Any], key: str, default: Any = UNKNOWN_VALUE) -> Any:
        """Safely extract a value from a dictionary."""
        return data.get(key, default)

    def safe_extract(key_path: list) -> Any:
        """Helper function to safely extract nested values using extract_value."""
        return extract_value(raw_data_artist_song_dict, key_path, UNKNOWN_VALUE)

    # Dictionary to store song information
    artist_song_data = {
        'genius_song_id': safe_get(raw_data_artist_song_dict, 'id'),
        'full_title': safe_get(raw_data_artist_song_dict, 'full_title'),
        'release_date': safe_get(raw_data_artist_song_dict, 'release_date'),
        'genius_album_id': safe_extract(['album', 'id']),
        'album_name': safe_extract(['album', 'name']),
        'genius_album_api_path': safe_extract(['album', 'api_path']),
        'genius_song_api_path': safe_get(raw_data_artist_song_dict, 'api_path'),
        'pageviews_genius': safe_extract(['stats', 'pageviews']),
        'genius_song_url': safe_get(raw_data_artist_song_dict, 'url'),
        'genius_song_image_url': safe_get(raw_data_artist_song_dict, 'song_art_image_url'),
        'language': safe_get(raw_data_artist_song_dict, 'language'),
        'genius_song_description': safe_extract(["description", "plain"]),
        'lyrics': safe_get(raw_data_artist_song_dict, 'lyrics'),
        'lyrics_is_complete': safe_get(raw_data_artist_song_dict, 'lyrics_state'),
    }

    # Extract primary artist's name safely
    primary_artist = safe_get(raw_data_artist_song_dict.get('primary_artist', {}), 'name', UNKNOWN_VALUE)
    artist_song_data['primary_artist'] = primary_artist

    # Extract other primary artists excluding the main primary artist
    primary_artists = safe_get(raw_data_artist_song_dict, 'primary_artists', [])
    artist_song_data['primary_artists'] = [
        artist.get('name', UNKNOWN_VALUE) for artist in primary_artists 
        if artist.get('name') != primary_artist
    ]

    # Extract featured artists
    artist_song_data['featured_artists'] = safe_get(raw_data_artist_song_dict, 'featured_artists', UNKNOWN_VALUE)

    return artist_song_data

In [7]:
def build_artist_info_genius(search_artist_result_dict: Dict[str, Any], UNKNOWN_VALUE: Any = "UNKNOWN") -> Dict[str, Any]:
    """
    Extracts and builds a dictionary containing artist information from the Genius API search result.

    Parameters:
    - search_artist_result_dict (Dict[str, Any]): The dictionary containing the search result from the Genius API.
    - UNKNOWN_VALUE (Any): The default value to use for missing keys. Defaults to "UNKNOWN".

    Returns:
    - Dict[str, Any]: A dictionary containing the extracted artist information.
    """
    artist_info = {
        'genius_artist_id': search_artist_result_dict.get('id', UNKNOWN_VALUE),
        'artist_name': search_artist_result_dict.get('name', UNKNOWN_VALUE),
        'genius_artist_api_path': search_artist_result_dict.get('api_path', UNKNOWN_VALUE),
        'genius_artist_url': search_artist_result_dict.get('url', UNKNOWN_VALUE),
        'genius_artist_image_url': search_artist_result_dict.get('image_url', UNKNOWN_VALUE),
        'genius_artist_description': extract_value(search_artist_result_dict, ["description", "plain"], UNKNOWN_VALUE),
        'twitter_name': search_artist_result_dict.get('twitter_name', UNKNOWN_VALUE),
        'facebook_name': search_artist_result_dict.get('facebook_name', UNKNOWN_VALUE),
        'instagram_name': search_artist_result_dict.get('instagram_name', UNKNOWN_VALUE),
        'is_verified_genius': search_artist_result_dict.get('is_verified', UNKNOWN_VALUE),
        'tracks': [build_artist_song_genius(song) for song in search_artist_result_dict.get('songs', [])]
    }

    return artist_info

In [8]:
artist_data = build_artist_info_genius(genius_search_artist_result_dict)

NameError: name 'genius_search_artist_result_dict' is not defined

In [255]:
with open(f"./assets/{artist_name}-{datetime.today().date()}-Genius.json", 'w') as f_in:
    json.dump(artist_data, f_in, indent=4)

In [10]:
with open(f"./assets/{artist_name}-{datetime.today().date()}-Genius.json", 'r') as f_out:
    artist = json.load(f_out)

FileNotFoundError: [Errno 2] No such file or directory: './assets/Zamdane-2024-10-18-Genius.json'

In [9]:
for track_num, track in enumerate(artist['tracks']):
    print(f"Track {track_num} - {track['full_title']}")

NameError: name 'artist' is not defined

In [14]:
dataset = pd.DataFrame(artist)

In [22]:
dataset.head(n = 10)

,genius_artist_id,artist_name,genius_artist_api_path,genius_artist_url,genius_artist_image_url,genius_artist_description,twitter_name,facebook_name,instagram_name,is_verified_genius,tracks
0,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 3128292, 'full_title': 'Fav..."
1,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 6541840, 'full_title': 'Aff..."
2,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 7135091, 'full_title': 'Boy..."
3,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 3611055, 'full_title': 'Dis..."
4,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 6893603, 'full_title': 'Le ..."
5,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 3313568, 'full_title': 'Règ..."
6,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 7640457, 'full_title': 'Vid..."
7,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 8891158, 'full_title': 'Aff..."
8,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 10084479, 'full_title': 'Mo..."
9,1159944,Zamdane,/artists/1159944,https://genius.com/artists/Zamdane,https://images.genius.com/2db9ead74b2f66a18bda...,"Zamdane, de son vrai nom Ayoub Zaidane, est un...",zamdane,zvmdvne,zamdane,True,"{'genius_song_id': 3611076, 'full_title': 'Qua..."


In [230]:
pd.DataFrame(list(dataset['tracks'].values))

,genius_song_id,full_title,release_date,genius_album_id,album_name,genius_album_api_path,genius_song_api_path,pageviews_genius,genius_song_url,genius_song_image_url,language,genius_song_description,lyrics,lyrics_is_complete,primary_artist,primary_artists,featured_artists
0,3128292,Favaro by Zamdane,2017-06-08,425938,Nova Grünt Tunes,/albums/425938,/songs/3128292,47099,https://genius.com/Zamdane-favaro-lyrics,https://images.genius.com/a6f7c834c0f61121da8a...,fr,Second morceau de l'artiste marseillais Zamdan...,"36 ContributorsFavaro Lyrics[Paroles de ""Favar...",complete,Zamdane,[],[]
1,6541840,Affamé #8 - Hayati by Zamdane,2021-03-03,728748,Affamé - Saison 2,/albums/728748,/songs/6541840,21299,https://genius.com/Zamdane-affame-8-hayati-lyrics,https://images.genius.com/73daff9eae6de9695697...,fr,?,18 ContributorsAffamé #8 - Hayati Lyrics[Parol...,complete,Zamdane,[],[]
2,7135091,Boyka by Zamdane (Ft. Dinos),2021-12-08,864511,Couleur de ma peine,/albums/864511,/songs/7135091,19777,https://genius.com/Zamdane-boyka-lyrics,https://images.genius.com/e494e1e555a57242f4ed...,fr,?,"32 ContributorsBoyka Lyrics[Paroles de ""Boyka""...",complete,Zamdane,[],"[{'api_path': '/artists/14806', 'header_image_..."
3,3611055,Disaster by Zamdane,2018-03-28,414601,20's,/albums/414601,/songs/3611055,17585,https://genius.com/Zamdane-disaster-lyrics,https://images.genius.com/b58f66f07ca79d120df5...,fr,"Premier extrait de “ 20’s ”, le premier EP de ...","13 ContributorsDisaster Lyrics[Paroles de ""Dis...",complete,Zamdane,[],[]
4,6893603,Le monde par ma fenêtre by Zamdane,2022-01-26,864511,Couleur de ma peine,/albums/864511,/songs/6893603,13511,https://genius.com/Zamdane-le-monde-par-ma-fen...,https://images.genius.com/d12628ac45c453238291...,fr,?,13 ContributorsLe monde par ma fenêtre Lyrics[...,complete,Zamdane,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,4566479,Freestyle Z #1 by Zamdane,2019-05-17,UNKNOWN,UNKNOWN,UNKNOWN,/songs/4566479,UNKNOWN,https://genius.com/Zamdane-freestyle-z-1-lyrics,https://images.genius.com/2db9ead74b2f66a18bda...,fr,?,3 ContributorsFreestyle Z #1 Lyrics[Couplet u...,complete,Zamdane,[],[]
113,5634312,L.A* by Zamdane,None,UNKNOWN,UNKNOWN,UNKNOWN,/songs/5634312,UNKNOWN,https://genius.com/Zamdane-la-lyrics,https://images.genius.com/2db9ead74b2f66a18bda...,fr,?,2 ContributorsL.A* LyricsCouplet 1 :\n\nJ'ai p...,complete,Zamdane,[],[]
114,8368731,Passage dans saison 3 épisode 3* by Zamdane (F...,None,UNKNOWN,UNKNOWN,UNKNOWN,/songs/8368731,UNKNOWN,https://genius.com/Zamdane-passage-dans-saison...,https://images.genius.com/2db9ead74b2f66a18bda...,fr,?,2 ContributorsPassage dans saison 3 épisode 3*...,complete,Zamdane,[],"[{'api_path': '/artists/12085', 'header_image_..."
115,10286853,From Time (Freestyle France Inter) by Zamdane,2024-04-15,UNKNOWN,UNKNOWN,UNKNOWN,/songs/10286853,UNKNOWN,https://genius.com/Zamdane-from-time-freestyle...,https://images.genius.com/2db9ead74b2f66a18bda...,fr,?,1 ContributorFrom Time (Freestyle France Inter...,complete,Zamdane,[],[]


In [88]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up client credentials
client_credentials_manager = SpotifyClientCredentials(os.environ['CLIENT_ID_SPOTIFY'], os.environ['CLIENT_SECRET_SPOTIFY'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [89]:
spotify_artist_search_result_dict = sp.search(artist_name, type='artist')

In [90]:
spotify_artist_data = {}

spotify_raw_data_artist_dict = extract_value(spotify_artist_search_result_dict,
                                             ['artists', 'items'],
                                             UNKNOWN_VALUE)[0]

In [91]:
def build_artist_info_spotify(spotify_raw_data_artist_dict: Dict[str, Any], UNKNOWN_VALUE: Any = "UNKNOWN") -> Dict[str, Any]:
    """
    Extracts and builds a dictionary containing artist information from the Spotipy API search result.

    Parameters:
    - spotify_raw_data_artist_dict (Dict[str, Any]): The dictionary containing the search result from the Spotipy
    - UNKNOWN_VALUE (Any): The default value to use for missing keys. Defaults to "UNKNOWN".

    Returns:
    - Dict[str, Any]: A dictionary containing the extracted artist information.
    """
    artist_info = {
        'artist_name': spotify_raw_data_artist_dict.get('name', UNKNOWN_VALUE),
        'artist_id_spotify': spotify_raw_data_artist_dict.get('id', UNKNOWN_VALUE),
        'artist_uri_spotify': spotify_raw_data_artist_dict.get('uri', UNKNOWN_VALUE),
        'artist_link_picture_spotify': extract_value(spotify_raw_data_artist_dict, ['images', 'url'], UNKNOWN_VALUE),
        'artist_genres_spotify': spotify_raw_data_artist_dict.get('genres', UNKNOWN_VALUE),
        'artist_popularity_spotify': spotify_raw_data_artist_dict.get('popularity', UNKNOWN_VALUE),
        'artist_related_artists_spotify': [related_artist_name['name'] for related_artist_name in sp.artist_related_artists(spotify_raw_data_artist_dict.get('id', UNKNOWN_VALUE)).get('artists', UNKNOWN_VALUE)],
        'artist_tracks': []
    }

    return artist_info

In [102]:
spotify_raw_data_artist_dict

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5CtPAGoxpJ4yLJLx6CSrO8'},
 'followers': {'href': None, 'total': 195082},
 'genres': ['cloud rap francais',
  'french hip hop',
  'pop urbaine',
  'rap conscient',
  'rap francais'],
 'href': 'https://api.spotify.com/v1/artists/5CtPAGoxpJ4yLJLx6CSrO8',
 'id': '5CtPAGoxpJ4yLJLx6CSrO8',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebc99fc3269d0996195e2308d6',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174c99fc3269d0996195e2308d6',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178c99fc3269d0996195e2308d6',
   'width': 160}],
 'name': 'Zamdane',
 'popularity': 60,
 'type': 'artist',
 'uri': 'spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8'}

In [103]:
spotify_artist_info = build_artist_info_spotify(spotify_raw_data_artist_dict)

In [104]:
spotify_artist_info

{'artist_name': 'Zamdane',
 'artist_id_spotify': '5CtPAGoxpJ4yLJLx6CSrO8',
 'artist_uri_spotify': 'spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8',
 'artist_link_picture_spotify': 'UNKNOWN',
 'artist_genres_spotify': ['cloud rap francais',
  'french hip hop',
  'pop urbaine',
  'rap conscient',
  'rap francais'],
 'artist_popularity_spotify': 60,
 'artist_related_artists_spotify': ['Lesram',
  'HOUDI',
  'So La Lune',
  'Rounhaa',
  'Dau',
  'Raplume',
  'Stony Stone',
  'Bekar',
  'J9ueve',
  'Wallace Cleaver',
  'Yvnnis',
  'Khali',
  'BEN plg',
  'Zed',
  'Jeune Mort',
  'ISHA',
  'La Fève',
  'BU$HI',
  'Luv Resval',
  'Prinzly'],
 'artist_tracks': []}

In [105]:
def get_artist_albums(artist_id: str, album_types=['album', 'single']) -> list:
    """
    Retrieves all albums (including singles) of the artist with the given artist_id.
    :param artist_id: Spotify artist ID
    :param album_types: List of album types to retrieve, default is ['album', 'single']
    :return: List of all albums (including singles)
    """
    albums = []
    results = sp.artist_albums(artist_id, album_type=','.join(album_types), limit=50)
    
    while results:
        albums.extend(results['items'])
        if results['next']:
            results = sp.next(results)
        else:
            break
    
    # Remove duplicates by checking album ID
    unique_albums = {album['id']: album for album in albums}.values()
    
    return list(unique_albums)

def get_album_tracks(album_id: str) -> list:
    """
    Retrieves all tracks from a specific album using album_id.
    :param album_id: Spotify album ID
    :return: List of tracks in the album
    """
    tracks = []
    results = sp.album_tracks(album_id)
    
    while results:
        tracks.extend(results['items'])
        if results['next']:
            results = sp.next(results)
        else:
            break
    
    return tracks

def get_all_tracks_of_artist(artist_id: str) -> list:
    """
    Extracts all tracks (singles and part of albums) for a given artist.
    :param artist_id: Spotify artist ID
    :return: List of all tracks from the artist
    """
    all_tracks = []
    albums = get_artist_albums(artist_id)
    
    for album in albums:
        album_id = album['id']
        tracks = get_album_tracks(album_id)
        all_tracks.extend(tracks)
    
    return all_tracks

#  # Replace with the artist's Spotify ID
tracks = get_all_tracks_of_artist(spotify_artist_info['artist_id_spotify'])

# Display the track names
for idx, track in enumerate(tracks, start=1):

    spotify_artist_info['artist_tracks'].append(
        {
        "track_id_spotify": track['id'],
        "track_name": track['name'],
        "track_uri_spotify": track['uri'],
        "track_audio_features_spotify": sp.audio_features(track['id']) 
    })


In [106]:
for track_num, track in enumerate(spotify_artist_info['artist_tracks']):

    print(f"Track {track_num} - {track['track_name']}")

Track 0 - Mouchkila
Track 1 - À ma guise
Track 2 - Le grand cirque
Track 3 - Million
Track 4 - Printemps
Track 5 - Alouette
Track 6 - Boboalam
Track 7 - Un peu de moi
Track 8 - Youm wara youm
Track 9 - Infini
Track 10 - Stylo magique
Track 11 - Loin d'ici
Track 12 - Mélancolie criminelle
Track 13 - Lalalala
Track 14 - Audi GT
Track 15 - Fleurs
Track 16 - Si on s'aimait
Track 17 - Noum
Track 18 - Étoiles dans les yeux
Track 19 - Monstres
Track 20 - Tout ce qu'il voulait
Track 21 - Le monde par ma fenêtre
Track 22 - Incomplet comme février
Track 23 - Boyka
Track 24 - 1,2,3,4
Track 25 - Stradivarius
Track 26 - Naïf
Track 27 - Fauves
Track 28 - Même à distance on est proches
Track 29 - Lettre à mon dieu
Track 30 - Foust L'Bando
Track 31 - Bonne année
Track 32 - C18
Track 33 - Zhar
Track 34 - Groupie love
Track 35 - Flouka
Track 36 - Libre comme une colombe
Track 37 - Vide quand t'es pas là
Track 38 - Affamé #6 - Vital
Track 39 - Affamé #7 - Histoires de la vraie vie
Track 40 - Affamé #8 - 

In [108]:
spotify_artist_song_data = pd.DataFrame(spotify_artist_info['artist_tracks'])

In [98]:
del spotify_artist_info['artist_tracks']
del spotify_artist_info['artist_related_artists_spotify']

In [109]:
spotify_artist_info

{'artist_name': 'Zamdane',
 'artist_id_spotify': '5CtPAGoxpJ4yLJLx6CSrO8',
 'artist_uri_spotify': 'spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8',
 'artist_link_picture_spotify': 'UNKNOWN',
 'artist_genres_spotify': ['cloud rap francais',
  'french hip hop',
  'pop urbaine',
  'rap conscient',
  'rap francais'],
 'artist_popularity_spotify': 60,
 'artist_related_artists_spotify': ['Lesram',
  'HOUDI',
  'So La Lune',
  'Rounhaa',
  'Dau',
  'Raplume',
  'Stony Stone',
  'Bekar',
  'J9ueve',
  'Wallace Cleaver',
  'Yvnnis',
  'Khali',
  'BEN plg',
  'Zed',
  'Jeune Mort',
  'ISHA',
  'La Fève',
  'BU$HI',
  'Luv Resval',
  'Prinzly'],
 'artist_tracks': [{'track_id_spotify': '0H8sSKJrB1xlmp5i0YgGrs',
   'track_name': 'Mouchkila',
   'track_uri_spotify': 'spotify:track:0H8sSKJrB1xlmp5i0YgGrs',
   'track_audio_features_spotify': [{'danceability': 0.6,
     'energy': 0.499,
     'key': 5,
     'loudness': -7.57,
     'mode': 0,
     'speechiness': 0.0364,
     'acousticness': 0.673,
     'instrumen

In [110]:
spotify_artist_profile_data = pd.DataFrame.from_dict(spotify_artist_info, orient="index").T

In [116]:
spotify_artist_profile_data

,artist_name,artist_id_spotify,artist_uri_spotify,artist_link_picture_spotify,artist_genres_spotify,artist_popularity_spotify,artist_related_artists_spotify,artist_tracks
0,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",[{'track_id_spotify': '0H8sSKJrB1xlmp5i0YgGrs'...


In [205]:
artist_spotify_information = spotify_artist_profile_data.drop(['artist_tracks'], inplace=False, axis = 1)
artist_spotify_songs = spotify_artist_profile_data[['artist_name', 'artist_id_spotify', 'artist_tracks']].explode('artist_tracks')

In [206]:
artist_spotify_information.reset_index(inplace=True, drop=True)

In [208]:
artist_spotify_information

,artist_name,artist_id_spotify,artist_uri_spotify,artist_link_picture_spotify,artist_genres_spotify,artist_popularity_spotify,artist_related_artists_spotify
0,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl..."


In [209]:
artist_spotify_songs.reset_index(inplace=True, drop = True)

In [210]:
artist_spotify_song_data = pd.DataFrame(list(artist_spotify_songs['artist_tracks']))

In [211]:
artist_spotify_song_data_audio_features = pd.DataFrame(list(artist_spotify_song_data['track_audio_features_spotify'].apply(lambda x : x[0] if x else {})))

In [213]:
artist_spotify_song_info = pd.concat([artist_spotify_song_data,artist_spotify_song_data_audio_features], axis=1).drop(['track_audio_features_spotify', 'type', 'id', 'uri', 'track_href', 'analysis_url'], inplace= False, axis = 1)

In [218]:
artist_spotify_song_info['artist_id_spotify'] =  artist_spotify_information['artist_id_spotify'].values[0]

In [219]:
artist_spotify_song_info

,track_id_spotify,track_name,track_uri_spotify,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,artist_id_spotify
0,0H8sSKJrB1xlmp5i0YgGrs,Mouchkila,spotify:track:0H8sSKJrB1xlmp5i0YgGrs,0.600,0.499,5,-7.570,0,0.0364,0.673,0.000000,0.2690,0.578,160.073,226413,4,5CtPAGoxpJ4yLJLx6CSrO8
1,1HaOYE4mMGG49HOf98uLTD,À ma guise,spotify:track:1HaOYE4mMGG49HOf98uLTD,0.706,0.618,4,-6.832,0,0.0828,0.634,0.000076,0.0907,0.349,137.988,161773,4,5CtPAGoxpJ4yLJLx6CSrO8
2,2lk8jLE8cF7NSL50pOXoTF,Le grand cirque,spotify:track:2lk8jLE8cF7NSL50pOXoTF,0.619,0.323,3,-7.642,0,0.0379,0.464,0.000041,0.2060,0.369,130.008,241760,5,5CtPAGoxpJ4yLJLx6CSrO8
3,33GQLlkZsYNw569Pr0ZAXw,Million,spotify:track:33GQLlkZsYNw569Pr0ZAXw,0.746,0.498,9,-9.168,0,0.0632,0.796,0.000000,0.0946,0.777,131.975,211493,4,5CtPAGoxpJ4yLJLx6CSrO8
4,5lV989DHPY9NO7ygRAAG2r,Printemps,spotify:track:5lV989DHPY9NO7ygRAAG2r,0.687,0.696,1,-6.757,0,0.0649,0.261,0.000000,0.1050,0.446,112.933,193373,4,5CtPAGoxpJ4yLJLx6CSrO8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,66PVjOeCXheSXKlFNtLjRR,Passe à l'appart,spotify:track:66PVjOeCXheSXKlFNtLjRR,0.867,0.508,1,-10.498,1,0.4590,0.520,0.000003,0.1480,0.404,135.153,184889,4,5CtPAGoxpJ4yLJLx6CSrO8
154,0z04vNP9kMa2ikRcue24gf,Dans L'mal,spotify:track:0z04vNP9kMa2ikRcue24gf,0.850,0.569,7,-9.594,1,0.4170,0.376,0.000004,0.0894,0.352,134.945,220444,4,5CtPAGoxpJ4yLJLx6CSrO8
155,5AERfJhWMvY6JFXsToXqIH,Tjpc,spotify:track:5AERfJhWMvY6JFXsToXqIH,0.783,0.524,10,-6.492,1,0.0685,0.510,0.000000,0.1110,0.191,125.044,200516,4,5CtPAGoxpJ4yLJLx6CSrO8
156,1Wi2mNxtNgkMuykhHudCPg,Règlement Freestyle #6,spotify:track:1Wi2mNxtNgkMuykhHudCPg,0.881,0.497,10,-10.349,1,0.1670,0.789,0.000000,0.1100,0.505,129.967,171692,4,5CtPAGoxpJ4yLJLx6CSrO8


In [225]:
pd.merge(artist_spotify_information, artist_spotify_song_info)

,artist_name,artist_id_spotify,artist_uri_spotify,artist_link_picture_spotify,artist_genres_spotify,artist_popularity_spotify,artist_related_artists_spotify,track_id_spotify,track_name,track_uri_spotify,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",0H8sSKJrB1xlmp5i0YgGrs,Mouchkila,spotify:track:0H8sSKJrB1xlmp5i0YgGrs,...,-7.570,0,0.0364,0.673,0.000000,0.2690,0.578,160.073,226413,4
1,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",1HaOYE4mMGG49HOf98uLTD,À ma guise,spotify:track:1HaOYE4mMGG49HOf98uLTD,...,-6.832,0,0.0828,0.634,0.000076,0.0907,0.349,137.988,161773,4
2,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",2lk8jLE8cF7NSL50pOXoTF,Le grand cirque,spotify:track:2lk8jLE8cF7NSL50pOXoTF,...,-7.642,0,0.0379,0.464,0.000041,0.2060,0.369,130.008,241760,5
3,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",33GQLlkZsYNw569Pr0ZAXw,Million,spotify:track:33GQLlkZsYNw569Pr0ZAXw,...,-9.168,0,0.0632,0.796,0.000000,0.0946,0.777,131.975,211493,4
4,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",5lV989DHPY9NO7ygRAAG2r,Printemps,spotify:track:5lV989DHPY9NO7ygRAAG2r,...,-6.757,0,0.0649,0.261,0.000000,0.1050,0.446,112.933,193373,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",66PVjOeCXheSXKlFNtLjRR,Passe à l'appart,spotify:track:66PVjOeCXheSXKlFNtLjRR,...,-10.498,1,0.4590,0.520,0.000003,0.1480,0.404,135.153,184889,4
154,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",0z04vNP9kMa2ikRcue24gf,Dans L'mal,spotify:track:0z04vNP9kMa2ikRcue24gf,...,-9.594,1,0.4170,0.376,0.000004,0.0894,0.352,134.945,220444,4
155,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",5AERfJhWMvY6JFXsToXqIH,Tjpc,spotify:track:5AERfJhWMvY6JFXsToXqIH,...,-6.492,1,0.0685,0.510,0.000000,0.1110,0.191,125.044,200516,4
156,Zamdane,5CtPAGoxpJ4yLJLx6CSrO8,spotify:artist:5CtPAGoxpJ4yLJLx6CSrO8,UNKNOWN,"[cloud rap francais, french hip hop, pop urbai...",60,"[Lesram, HOUDI, So La Lune, Rounhaa, Dau, Rapl...",1Wi2mNxtNgkMuykhHudCPg,Règlement Freestyle #6,spotify:track:1Wi2mNxtNgkMuykhHudCPg,...,-10.349,1,0.1670,0.789,0.000000,0.1100,0.505,129.967,171692,4
